In [ ]:
# default_exp core

# sniptool

> API details.

In [ ]:
#hide
# from nbdev.showdoc import *

In [19]:
#exporti
"""
%snip magic for snippets management, supported by a sqlite db.

%d        -- for object inspection
%tidy     -- tidy-up notebook
%g        -- git commands
%saveimg  -- save cell outputs to disk
%lint     -- save cell outputs to disk

%%toc     -- create toc divider
%%runingb -- running in the background

snippets naming: # @desc #t1 #t2

todo
- [x] yaml cfg
- [x] absolute path for db
- [x] reset
- [x] shortcut
- [x] setup
- [ ] savefig, uuid
- [ ] url to source file
- [ ] pre-commit, not ipynb, only select ipynb py and images, update images with uuid
https://gist.github.com/tylerneylon/697065ca5906c185ec6dd3093b237164
"""

# Author: Jun Xiong. 04/08/2022, 12:46
# Distributed under the terms of the Modified BSD License.

import base64
import os
import sys
import json
import pandas as pd
import yaml
import logging
import pycodestyle as pycodestyle_module
import tempfile
from glob import glob
from PIL import Image
from io import BytesIO, StringIO
from os.path import expanduser
from pathlib import Path
from pygments import highlight
from pygments.lexers import PythonLexer
from pygments.formatters import HtmlFormatter
from IPython.core.display import HTML, display
from IPython.core.magic import Magics, magics_class, line_magic, cell_magic
from sqlalchemy import create_engine, Column, String, Float, Text
from sqlalchemy.dialects.sqlite import insert, dialect
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()


class NBfiles(Base):
    __tablename__ = "nbfiles"
    path = Column(String(256), primary_key=True)
    mtime = Column(Float(64))


class Snippets(Base):
    __tablename__ = "snippets"
    tags = Column(String(256))
    desc = Column(String(256), primary_key=True)
    source = Column(Text)
    url = Column(String(256))


def nb_meta_parse(path):
    snippets = []
    notebook = json.load(open(path))

    for cell in notebook["cells"]:
        if not cell["source"]:
            continue
        head = cell["source"][0]
        if head.startswith('# @'):
            desc, *tags = tuple(head.lstrip('# @').split('#'))
#             print(desc, tags)
            tags = ";".join(tags) if tags else ''
            source = "".join(cell["source"][1:])
            url = ""
            snippets.append(
                dict(
                    tags=tags,
                    desc=desc,
                    source=source,
                    url=url,
                )
            )
    return len(snippets) > 0, pd.DataFrame(snippets)


def get_file_list(targets, exclude=['__pycache__', '.ipynb_checkpoints']):
    """"
    This function recursively prints all contents of a pathlib.Path object
    """
    import configparser
    nb_list = [(None, None)]
    for target in targets:
        target = Path(target)
        cfg1, cfg2 = target / 'ghconfig', target / '.git/config'
        if cfg1.is_file():  # has ghconfig
            cfg = yaml.load(open(str(cfg1)), Loader=yaml.BaseLoader)
            root = cfg['repo'].rstrip('.git') + '/' + target.name
        elif cfg2.is_file():
            config = configparser.ConfigParser()
            config.read(str(cfg2))
            root = config['remote "origin"']['url'].rstrip(
                '.git') + '/' + target.name
        else:
            root = None
        for e in exclude:
            if target.name.find(e) != -1:
                return list(map(list, zip(*nb_list)))
        for file in target.iterdir():
            if file.is_dir():
                if '.' in file.name:
                    continue
                nb_list += get_file_list([file.absolute().as_posix()],
                                         exclude=exclude)
            elif file.name.endswith('.ipynb'):
                nb_list.append((file.absolute().as_posix(), root))
    # remove invalid lines
    nb_list = list(filter(lambda x: x[1], nb_list))
    return list(map(list, zip(*nb_list)))
# get_file_list(['/home/jovyan/helper'], exclude=['.ipynb_checkpoints'])

def compile_query(query):
    compiler = (
        query.compile if not hasattr(
            query, "statement") else query.statement.compile
    )
    return compiler(dialect=dialect())

In [ ]:
#exporti
@magics_class
class SnippetsMagics(Magics):
    """snip magic for snippets management.
    Provides the %snip magic.
    """
    def __init__(self, shell):
        super(SnippetsMagics, self).__init__(shell=shell)
        dotfile='~/config.yaml'
        cfg = yaml.load(open(expanduser(dotfile)), Loader=yaml.BaseLoader)
        self.db = cfg['config_dir']+"/SnippetDB.db"
        self.search_dirs = cfg['search_dirs']
        self.engine = create_engine(f"sqlite:///{self.db}")
        Base.metadata.create_all(self.engine)
        self.snippets = pd.read_sql("snippets", self.engine)

    def get_snippets_file_list(self):
        #         if os.path.isfile(cfg):
        include = [d for d in self.search_dirs if not d.startswith("!")]
        exclude = [d[1:] for d in self.search_dirs if d.startswith("!")]
        nb_list, links = get_file_list(include, exclude)
#         print(nb_list)
#         mtime = [os.path.getmtime(path) for path in search_list]
        mtime = list(map(os.path.getmtime, nb_list))
#         print(mtime, nb_list, links)
        return pd.DataFrame(zip(nb_list, mtime, links), columns=["path", "mtime", "url"])

    def upsert(self, df, model):
        table = model.__table__
        stmt = insert(table).values(df.to_dict(orient="records"))
        update_cols = [
            c.name for c in table.c if c not in list(table.primary_key.columns)
        ]
        on_conflict_stmt = stmt.on_conflict_do_update(
            index_elements=table.primary_key.columns,
            set_={c.key: c for c in stmt.excluded if c.key in update_cols},
        )
        self.engine.execute(on_conflict_stmt)

    def update(self):
        nb_to_search = self.get_snippets_file_list()
        try:
            nb_exists = pd.read_sql("nbfiles", self.engine).rename(
                columns={"mtime": "utime"}
            )
            nb_to_update = nb_exists.merge(nb_to_search, how="right")
            nb_to_update = nb_to_update.fillna(0)
        except RuntimeError:
            nb_to_update = nb_exists
        # print(nb_to_update, self.nb_exists, self.nb_to_search)
        nb_to_update = nb_to_update.assign(
            isnew=nb_to_update.mtime > nb_to_update.utime + 1)
#         nb_to_update.loc[:, "isnew"] = nb_to_update.apply(
#             lambda r: r["mtime"] > r["utime"] + 1, axis=1
#         )
        for path in nb_to_update[nb_to_update.isnew].path.values:
            #             print(path)
            ok, new_snippets = nb_meta_parse(path)
            if ok:
                print(
                    f"updating ({new_snippets.shape[0]}) snippets from {path}")
                self.upsert(new_snippets, Snippets)
        new_db = nb_to_update[nb_to_update.isnew]
        if not new_db.empty:
            self.upsert(new_db[["path", "mtime"]], NBfiles)
        self.snippets = pd.read_sql("snippets", self.engine)

    def reset(self):
        self.engine.dispose()
        os.remove(self.db)
        self.engine = create_engine(f"sqlite:///{self.db}")
        Base.metadata.create_all(self.engine)
        self.update()

    def info(self):
        snippets = pd.read_sql("snippets", self.engine)
        nb_exists = pd.read_sql("nbfiles", self.engine)
        print(
            "{} snippets, {} notebooks in SnippetDB".format(
                snippets.shape[0], nb_exists.shape[0]
            )
        )

    def search(self, q):
        if q == "*":
            return self.snippets
        return self.snippets.query(f"desc.str.contains('{q}')", engine="python")

    def load(self, q, pos=0):
        if type(q) == list:
            q = " ".join(q)
        results = self.search(q)
        self.shell.set_next_input(results.iloc[pos]["source"], replace=False)

    def disp(self, q, short=False):
        display(
            HTML(
                """
        <style>
        {pygments_css}
        </style>
        """.format(
                    pygments_css=HtmlFormatter().get_style_defs(".highlight")
                )
            )
        )
        snippets = self.search(q).reset_index()
        for _, row in snippets.iterrows():
            if short:
                content = row["desc"]
            else:
                content = row["desc"] + '\n' + row["source"][:80]
            display(
                HTML(
                    data=highlight(f"({_})" + content,
                                   PythonLexer(), HtmlFormatter())
                )
            )

    @line_magic
    def snip(self, parameter_s=""):
        """snippets management

        Usage:
        * ``%snip``          - total numbers of all snippets and notebooks
        * ``%snip bar``      - search in the db
        * ``%snip -r``       - reset indexing
        * ``%snip -u``       - update db
        * ``%snip -s``       - short output

        New search directories can be inserted into config.yaml['search_dirs']
        """

        opts, argsl = self.parse_options(parameter_s, "rus", mode="string")
        args = argsl.split()
        if "r" in opts:
            self.reset()
        elif "u" in opts:
            self.update()
        elif not args:
            self.info()
        elif args[-1].isdigit():  # load snippets
            pos = int(args[-1])
            self.load(args[:-1], pos=pos)
        # default action - disp the snippets
        else:
            self.disp(" ".join(args), short='s' in opts)

    @line_magic
    def d(self, parameter_s="", n_cols=5, regex='', cap=False):
        import inspect

        def chunks(names: list, n: int):
            """Yield successive n-sized chunks from lst."""
            r = int(len(names) / n + 0.5)
            for i in range(0, len(names), r):
                yield names[i: i + r]

        def output(items, n_cols=5):
            newcolumns = list(chunks(items, n_cols))
            n_rows, n_columns = len(newcolumns[0]), len(newcolumns)
            newcolumns[-1].extend([""] * (n_rows * n_columns - len(items)))
            array_outputs = [[row[i] for row in newcolumns]
                             for i in range(n_rows)]
            col_width = (
                max(len(word) for row in array_outputs for word in row) + 2
            )  # padding
            for row in array_outputs:
                print("".join(word.ljust(col_width) for word in row))
        opts, argsl = self.parse_options(parameter_s, "cn:s:", mode="string")
        args = argsl.split()
        if 'n' in opts:
            n_cols = int(opts['n'])
        if 'c' in opts:
            cap = True
        if len(args) > 1:
            regex = args[1]
        for _, f in enumerate(inspect.stack()[4:]):
            if args[0] in f[0].f_locals:
                target = f[0].f_locals[args[0]]
                items = list(filter(lambda x: x[0] != "_", dir(target)))
                if regex:
                    items = list(filter(lambda x: x.find(regex) != -1, items))
                if not cap:
                    items = list(filter(lambda x: x[0].islower(), items))
                output(items, n_cols=n_cols)
                break

    @line_magic
    def lint(self, parameter_s=""):
        opts, argsl = self.parse_options(parameter_s, "abf", mode="string")
        args = argsl.split()
        assert os.path.isfile(args[0])
        if "f" in opts:
            cmd = f'autopep8 --in-place {args[0]}'
            print('autopep8: finished')
        elif "b" in opts:
            cmd = f'nbblack {args[0]}'
        elif "a" in opts:
            cmd = f'pylint --disable=R,C --rcfile ~/.pylintrc {args[0]}'
        else:
            cmd = f'pycodestyle --show-source --ignore E501 {args[0]}'
        print(cmd)
        os.system(cmd)

    @cell_magic
    def runingb(self, line, codes):
        from multiprocessing import Process
        from tempfile import NamedTemporaryFile

        def wrapper(pyfile):
            import subprocess
            import time
            start = time.time()
            proc = subprocess.Popen(['python', pyfile],
                                    stdout=subprocess.PIPE)
            print(proc.stdout.readline().decode())
            print("Wall time:", time.time() - start)
        f = NamedTemporaryFile(mode='w', suffix='.py', delete=False)
#         print(f.name)
        with open(f.name, 'w') as fout:
            fout.write(codes)
        p = Process(target=wrapper, args=(f.name,), name='wrapper')
        p.start()

    @cell_magic
    def toc(self, ipynb, markdown_content):
        def mkreader(md):
            for line in ("\n" + md).split("\n#"):
                if line:
                    yield ("#" + line)

        cells = json.load(open(ipynb))
        for source in reversed(list(mkreader(markdown_content))):
            cell = {
                "cell_type": "markdown",
                "id": "",
                "metadata": {},
                "source": [line + "\n" for line in source.splitlines()],
            }
            cells["cells"].insert(0, cell)
        with open(ipynb, "w") as fout:
            json.dump(cells, fout, indent=2)

    @line_magic
    def tidy(self, ipynb: str):
        cells = json.load(open(ipynb))
        cnt = 1
        clean_version = []
        for cell in cells["cells"]:
            if cell["cell_type"] == "code":
                if not cell["source"]:
                    continue
                if not cell["source"][0].startswith("#"):
                    continue
                cell["execution_count"] = None
            elif cell["cell_type"] == "markdown":
                if not cell["source"]:
                    continue
                headline = cell["source"][0]
                if headline.startswith("#") or headline.startswith("<a id="):
                    if cnt == 1:
                        print("# TOC")
                    header = headline.lstrip("# ").strip()
                    print(f"* [{header}](#cell{cnt})")
                    cell["source"].insert(1, f'<a id="cell{cnt}"></a>\n')
                    cnt += 1
            clean_version.append(cell)
        out_file = open(ipynb, "w")
        cells["cells"] = clean_version
        json.dump(cells, out_file, indent=2)
        out_file.close()

    @line_magic
    def g(self, parameter_s: str):
        opts, argsl = self.parse_options(parameter_s, "drz", mode="string")
        args = argsl.split()
        ignore = '.ipynb_checkpoints/ ghconfig **/__pycache__ **/data/ **/obsolete'.replace(
            ' ', '\n')
        if not args:
            os.system("git status")
        else:
            cmd, opts = args[0], " ".join(args[1:])
            assert os.path.isfile("ghconfig")
            cred = yaml.load(open("ghconfig"), Loader=yaml.BaseLoader)
            url = f'{cred["token"]}@{cred["repo"].replace("https://","")}'
            if cmd == "get":
                pass
            elif cmd == "push":
                os.system(f"git push {url} {opts}")
            elif cmd == "pull":
                os.system(f"git pull {url} {opts}")
            elif cmd == "loc":
                print(cred["repo"])
            elif cmd == "save":
                if not opts:
                    opts = "update"
                os.system(f'git add -u && git commit -m "{opts}"')
            else:
                os.system(f"git {cmd} {opts}")

            if cmd == "init":
                os.system("mkdir -p obsolete")
                with open(".gitignore", "w") as f:
                    f.write(ignore)

    @line_magic
    def saveimg(self, parameter_s: str, folder='img'):
        def clip_output_img(path):
            assert os.path.isfile(path)
            with open(path) as fin:
                notebook = json.load(fin)
                for cell in notebook["cells"]:
                    if not cell["source"]:
                        continue
                    head = cell["source"][0]
                    if head.startswith('# !'):
                        oname = folder + '/' + head.lstrip('# !').strip()
                        img_png = cell['outputs'][0]['data']['image/png']
                        im = Image.open(BytesIO(base64.b64decode(img_png)))
                        im.save(oname, 'PNG')
                        print(f'-- {oname} created.')
        opts, argsl = self.parse_options(parameter_s, "a", mode="string")
        args = argsl.split()
        if not os.path.exists(folder):
            os.mkdir(folder)
        if not args:  # * all ipynb
            args = glob('*.ipynb')
            for path in args:
                print(path)
                if 'a' in opts:
                    clip_output_img(path)

        else:
            for path in args:
                clip_output_img(path)

    @cell_magic
    def pep8(self, line, cell, auto=False):
        """pycodestyle cell magic for pep8"""
        if cell.startswith(('!', '%%', '%')):
            return
        with tempfile.NamedTemporaryFile(mode='r+', delete=False) as f:
            f.write('# The %%pycodestyle cell magic was here\n' + cell)
            f.flush()
            f.close()
        format = '%(row)d:%(col)d: %(code)s %(text)s'
        pycodestyle = pycodestyle_module.StyleGuide(format=format)
        pcs_result = pycodestyle.check_files(paths=[f.name])
        try:
            os.remove(f.name)
        except OSError as e:  # if failed, report it back to the user ##
            logger.error("Error: %s - %s." % (e.filename, e.strerror))


In [9]:
!nbdev_build_lib
!nbdev_build_docs

Converted 00_core.ipynb.
Converted index.ipynb.
converting: /home/jovyan/guide/sniptool/00_core.ipynb
converting /home/jovyan/guide/sniptool/index.ipynb to README.md


In [13]:
%reload_ext sniptool

In [16]:
%g add .
%g save
%g push

AssertionError: 

In [11]:
!pip install -qe .

In [10]:
%load_ext sniptool

In [11]:
%snip??

Source:
    @line_magic
    def snip(self, parameter_s=""):
        """snippets management

        Usage:
        * ``%snip``          - total numbers of all snippets and notebooks
        * ``%snip bar``      - search in the db
        * ``%snip -r``       - reset indexing
        * ``%snip -u``       - update db
        * ``%snip -s``       - short output

        New search directories can be inserted into config.yaml['search_dirs']
        """

        opts, argsl = self.parse_options(parameter_s, "rus", mode="string")
        args = argsl.split()
        if "r" in opts:
            self.reset()
        elif "u" in opts:
            self.update()
        elif not args:
            self.info()
        elif args[-1].isdigit():  # load snippets
            pos = int(args[-1])
            self.load(args[:-1], pos=pos)
        # default action - disp the snippets
        else:
            self.disp(" ".join(args), short='s' in opts)
File:   ~/guide/sniptool/sniptool/core.py


In [12]:
%g

On branch master
Your branch is up to date with 'origin/master'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   00_core.ipynb
	modified:   settings.ini

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	Makefile
	docs/_config.yml
	docs/_data/
	sniptool/

no changes added to commit (use "git add" and/or "git commit -a")
